In [1]:
import hashlib
import binascii

# Zad 1

In [2]:
msg = "Kot".encode('utf8')

## MD5

In [3]:
binascii.hexlify(hashlib.md5(msg).digest())

b'c0d03d2d3e717da54ffdfc8a76c0f089'

## SHA-1

In [4]:
binascii.hexlify(hashlib.sha1(msg).digest())

b'a0e5cd812455e04d6e33646cd8dc17e05b674231'

## SHA-2

In [5]:
binascii.hexlify(hashlib.sha224(msg).digest())

b'bacfd6a442d02b096f13baa3e2e11b1ca81e702351f0d3289b18592d'

In [6]:
binascii.hexlify(hashlib.sha256(msg).digest())

b'aedaac3e798149ebaec99435ea67f2ff1fc8b5cd2f3b039b885bdf8c04678c03'

In [7]:
binascii.hexlify(hashlib.sha384(msg).digest())

b'2c4cdb621cf134cc6d04c76012b3f8c06247463c95ce9f6158a8b2218f2e9ab323e111551c494e8b92bb95955df3913d'

In [8]:
binascii.hexlify(hashlib.sha512(msg).digest())

b'71062884a3882e5442aa9acaff2b570c3470431f60e868a4178cbd0460f71ac0d5a679f4647003d5b683a7c9091a49b1075c72dc63bbbc9aa461c3af9129ddd9'

## SHA-3

In [9]:
binascii.hexlify(hashlib.sha3_224(msg).digest())

b'b6dd6f2e4082800cea403f7e387f80e2be1ee0ad6860ea5f6843dd45'

In [10]:
binascii.hexlify(hashlib.sha3_256(msg).digest())

b'16c6f78ba37ae968b2602249278aad82aea653662b3e9583d598030f0fef5c4d'

In [11]:
binascii.hexlify(hashlib.sha3_384(msg).digest())

b'8e24b4f5a8c62208ee9161f4ff8bcfd9d0aced7a0ee8091ff74abf3f2058112c48d10e4fc6b4b037055579789ab18404'

In [12]:
binascii.hexlify(hashlib.sha3_512(msg).digest())

b'4e8c7f92a8e50c690ebfc01aca3e91d29fae439427aa979271d159f4a0bc4f741b3bab6144ecf36cacf21e23aa1720ce952fba1b8836dd7c819e4c6d73604b4a'

# Zad 2

In [13]:
import secrets
import string
import random
from time import time
import numpy as np

In [14]:
alphabet = string.ascii_letters + string.digits

In [15]:
passwords = [''.join(secrets.choice(alphabet) for i in range(x)).encode('windows-1250') for x in random.sample(list(range(10, 100)) * 100_000, 100_000)]

In [16]:
functions = [
    ('md5', hashlib.md5),
    ('sha-1', hashlib.sha1),
    ('sha-224', hashlib.sha224),
    ('sha-256', hashlib.sha256),
    ('sha-384', hashlib.sha384),
    ('sha-512', hashlib.sha512),
    ('sha-3-224', hashlib.sha3_224),
    ('sha-3-256', hashlib.sha3_256),
    ('sha-3-384', hashlib.sha3_384),
    ('sha-3-512', hashlib.sha3_512)
]

In [17]:
result = []

for function in functions:
    start = time()
    hashes = [function[1](x) for x in passwords]
    stop = time()
    hashes_lengths = np.array([len(binascii.hexlify(x.digest())) for x in hashes])
    result.append([
        function[0],
        (stop - start) / len(hashes),
        hashes_lengths.mean().item(),
        hashes_lengths.std().item(),
        np.median(hashes_lengths).item()
    ])

In [18]:
from tabulate import tabulate

print(tabulate(result, headers=['name', 'mean time', 'mean len', 'len std dev', 'len median']))

name         mean time    mean len    len std dev    len median
---------  -----------  ----------  -------------  ------------
md5        3.7066e-07           32              0            32
sha-1      4.59905e-07          40              0            40
sha-224    4.60117e-07          56              0            56
sha-256    4.6998e-07           64              0            64
sha-384    4.90549e-07          96              0            96
sha-512    5.30007e-07         128              0           128
sha-3-224  6.00731e-07          56              0            56
sha-3-256  6.2988e-07           64              0            64
sha-3-384  6.31423e-07          96              0            96
sha-3-512  7.03094e-07         128              0           128


# Zad 3

In [19]:
hash_md5 = hashlib.md5(b'test')
print(binascii.hexlify(hash_md5.digest()))

b'098f6bcd4621d373cade4e832627b4f6'


Strona https://md5decrypt.net/en/ pozwala zdekodować hash i faktycznie zwraca wartość wejściową "test", więc wartość musi być powrzechnie znana, albo łatwo ją odgadnąć. Użycie dłuższej wartości wejściowej niepozwala tego powtórzyć.

Krótkie hasła można odkryć poprzez wygenerowanie ich hashy i porównanie z tymi w bazie.

# Zad 4

Funkcja MD5 nie jest już uznawana za bezpieczną. Możliwość szybkiego znajdowania kolizji (różnych słów dających ten sam skrót) sprawia, że MD5 nie jest zalecane do ochrony danych.

# Zad 5

In [55]:
from collections import Counter

In [22]:
hashes = [hashlib.sha256(x).digest() for x in passwords]

In [62]:
def first12bits(val):
    return (val[0] << 2) | (val[1] >> 6)

In [63]:
counter = Counter()

for h in hashes:
    h12 = first12bits(h)
    counter[h12] += 1

In [94]:
total_collisions = sum(1 for count in counter.values() if count > 1)

values = sorted([(bin(x[0]).replace('0b', '').rjust(12, '0'), x[0], x[1]) for x in counter.items() if x[1] > 1], key=lambda x: -x[2])

headers = [
    "12 bits",
    "12 bits int",
    "collisions"
]

print(f"total collisions: {total_collisions}\n")
print(tabulate(values, headers=headers))

total collisions: 1024

     12 bits    12 bits int    collisions
------------  -------------  ------------
000111001110            462           128
001100100100            804           128
000110011111            415           128
001111000100            964           128
001101000110            838           128
000010100110            166           126
001011001100            716           126
000100110110            310           125
000101111111            383           124
001110000110            902           123
000010001110            142           122
000110011100            412           122
001111010011            979           121
000101100101            357           121
001110111101            957           121
000100110000            304           121
001101101100            876           121
001001010010            594           121
000110010100            404           120
001011111111            767           120
001101111110            894           120
0001011101

# Zad 6

In [189]:
def change_1bit(val):
    res = bytearray(val)
    pos = random.randint(0, len(res) - 1)
    bitpos = random.randint(0, 7)
    res[pos] = res[pos] ^ (1 << bitpos)
    return bytes(res)

In [223]:
hashes = [(hashlib.sha256(x).digest(), hashlib.sha256(change_1bit(x)).digest()) for x in passwords]

In [245]:
changes = []
total_bit_count = len(hashes[0][0]) * 8

for (h1, h2) in hashes:
    changes.append(sum([(x ^ y).bit_count() for (x, y) in zip(h1, h2)]))

changes = np.array(changes) / total_bit_count

result = [
    changes.mean(),
    changes.std(),
    np.median(changes)
]

headers = [
    "avg",
    "std",
    "median"
]

print(f'total bit count = {total_bit_count}')
print(tabulate([result], headers=headers))

total bit count = 256
     avg        std    median
--------  ---------  --------
0.499993  0.0312092       0.5


## Wyniki

Powyższa tabelka pokazuje, że dla funkcji SHA-256 wszystkie bity wyjściowe powinnyzmienić się z prawdopodobieństwem 0.5, ponieważ średnie prawdopodbieństwo zmiany bitów nie odbiega od tej wartości, odchylenie standardowe jest bardzo małe a mediana wynosi tyle ile powinna. Testy przeprowadzone dla $n$ ciągów o różnej długości, gdzie $n = 100\,000$.